**Approach: Simple Query + Word2Vec + Rochio + gamma= 0**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Reading csv files containing tokens of documents and queries

In [0]:
import pandas as pd
df=pd.read_csv('drive/My Drive/dataset.csv')

In [0]:
import pandas as pd
dfq=pd.read_csv('drive/My Drive/dataset_query.csv')

In [0]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

Creating list of documents

In [0]:
sent=[]
for i in range(125511):
  if(i%10000==0):
    print(i)
  tokens=df["Tokens"][i][1:len(df["Tokens"][i])-2]
  tokens_list=tokens.split(', ')
  for i in range(len(tokens_list)):
    tokens_list[i]=tokens_list[i][1:len(tokens_list[i])-1]
  sent.append(tokens_list)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [0]:
print(len(sent))

125511


**Word embedding using SkipGram Technique**

Window size is taken as 2 , Dimension: 100

In [0]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
model = Word2Vec(sent, size=100, window=2, min_count=1,sg=1) #using skipgram technique

In [0]:
model.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model=Word2Vec.load("drive/My Drive/word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model.train(sent,total_examples=125511,epochs=4)

(129914225, 131597408)

Creating list of queries

In [0]:
sent_q=[]
for i in range(50):
  if(i%5==0):
    print(i)
  tokens=dfq["Tokens"][i][1:len(dfq["Tokens"][i])-1]
  tokens_list=tokens.split(', ')
  for i in range(len(tokens_list)):
    tokens_list[i]=tokens_list[i][1:len(tokens_list[i])-1]
  sent_q.append(tokens_list)

0
5
10
15
20
25
30
35
40
45


In [0]:
model.wv["reason"] #Sample word vector of 100 dimensions

array([ 0.36243615,  0.04503331, -0.43213254,  0.09851836,  0.5906064 ,
        0.5114799 , -0.12560925,  0.25110355,  0.60464567, -0.7958527 ,
       -0.21644698, -0.9304483 ,  0.05804143,  0.5504083 , -0.00342163,
        0.06829119,  0.22590208,  0.0411131 ,  0.03289723, -0.20925708,
        0.11944792,  0.22260585,  0.06296273, -0.24071032, -0.01663958,
       -0.0427087 , -0.4999284 , -0.02924724, -0.24229896,  0.5857353 ,
        0.13247328,  0.17263772,  0.28781608,  0.25913167, -0.01311486,
       -0.26647222,  0.0933061 ,  0.39802426, -0.01516036, -0.5227983 ,
        0.22576255, -0.34148163, -0.56822526, -0.06391629,  0.51439875,
       -0.25317022, -0.36146206, -0.44514993,  0.1182727 , -0.06484474,
        0.09106957, -0.50062484, -0.54174817,  0.07958966,  0.17889644,
       -0.6904657 , -0.25952485,  0.06954725,  0.04208698,  0.44061315,
        0.38641182,  0.09739266,  0.05731147, -0.1594837 ,  0.61802554,
        0.65077263, -0.5124919 , -0.2432941 ,  0.59344745, -0.49

Calculating word2vec embedding (vectors) for each doc

In [0]:
# calculating vectors for each doc

import numpy as np
doc_rep=np.zeros((125511,100),dtype='float32')
for i in range(125511):
  docs=sent[i]
  no_of_words=len(docs)
  word_rep=[]
  for x in docs:
    word_rep.append(model.wv[x]);
  word_rep_arr=np.array(word_rep)
  if(i%10000==0):
    print(word_rep_arr.shape)
    print(i)
  doc_rep[i]=np.average(word_rep_arr,axis=0)


(111, 100)
0
(177, 100)
10000
(167, 100)
20000
(178, 100)
30000
(157, 100)
40000
(79, 100)
50000
(280, 100)
60000
(170, 100)
70000
(138, 100)
80000
(303, 100)
90000
(98, 100)
100000
(199, 100)
110000
(411, 100)
120000


In [0]:
print(doc_rep.shape)

(125511, 100)


Calculating word2vec embedding (vectors) for each query

In [0]:
# calculating vectors for each query 
import numpy as np
query_rep=np.zeros((50,100),dtype='float32')
p=0;
for i in range(50):
  #if(i==11 or i==25):
   # continue
  query=sent_q[i]
  no_of_words=len(query)
  q_rep=[]
  for x in query:
    q_rep.append(model.wv[x]);
  q_rep_arr=np.array(q_rep)
  print(i)
  query_rep[p]=np.average(q_rep_arr,axis=0)
  p=p+1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [0]:
print(query_rep.shape)

(50, 100)


Calculating Cosine Similarity distance

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(query_rep,doc_rep)

In [0]:
import pandas as pd
df3=pd.read_csv('drive/My Drive/file_names.csv')

import pandas as pd
df4=pd.read_csv('drive/My Drive/relevant_docs.csv')

Note: **Threshold has been taken as 0.83**

In [0]:
dist=[]
rel_docs=[] # to store relev
actual_rel_docs=[] # actual rel
rel_docs_ids=[] # to store indices of rel doc vectors
p=0;
for k in range(0,50):
  dist=[]
  a=[] # to store relev
  b=[] # actual rel
  ai=[] # to store indices of relev
  for i in range(len(cos_lib[k])):
    dist.append((i,cos_lib[k][i]))

  dist=sorted(dist,key=lambda x:x[1],reverse=True)
  print("Processing...");
  i=0;
  while(dist[i][1]>=0.83 and i<125511):
   # print(dist[i]," ",df3["File_Name"][dist[i][0]].split('\\')[-1])
    a.append(df3["File_Name"][dist[i][0]].split('\\')[-1])
    ai.append(dist[i][0])
    i=i+1;
  
  print("-----------")
  
  print("Actual Relevant docs: ")
  mdf=(df4[df4["Id"]==k+76])
  #print(mdf)
  #print(len(mdf))
  
  for i in range(len(mdf)):
    b.append(mdf["Doc_Name"][p])
    #print(mdf["Doc_Name"][p])
    p=p+1;
  rel_docs.append(a)
  actual_rel_docs.append(b) # actual rel
  rel_docs_ids.append(ai)
  print("------------------------------------")

Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
-----

In [0]:
matches=0
for i in range(50):
  print(len(actual_rel_docs[i])," ",len(set(rel_docs[i]) & set(actual_rel_docs[i])))

5   4
20   4
3   2
4   0
8   2
6   5
23   5
9   3
3   1
22   10
7   6
9   6
22   19
47   14
5   5
12   1
38   0
19   13
25   0
11   1
3   2
22   8
5   0
3   0
21   6
13   9
17   0
7   1
8   2
13   12
4   0
14   10
11   5
10   5
11   2
16   0
17   0
14   0
8   3
19   1
10   1
1   1
10   7
7   0
12   8
21   0
20   3
19   1
7   1
13   0


In [0]:
matches=0
for i in range(50):
  matches=matches+len(set(rel_docs[i]) & set(actual_rel_docs[i]))

print("Total Matches: ",matches)

Total Matches:  191


In [0]:
print(list(set(rel_docs[0]) & set(actual_rel_docs[0])))

['1070602_nation_story_7865940.utf8', '1070603_nation_story_7869357.utf8', '1070611_nation_story_7906812.utf8', '1040901_nation_story_3702283.utf8']


In [0]:
import pandas as pd
df5=pd.read_csv('drive/My Drive/file_names_new.csv')

Storing Relevant and Non-Relevant vectors for each query

In [0]:
dr_rev=np.zeros((50,100),dtype='float32')
ndr_rev=np.zeros((50,100),dtype='float32')
for i in range(50):
  dr=[]
  ndr=[]
  print("processing")
  matched_files=list(set(rel_docs[i]) & set(actual_rel_docs[i]))
  for k in range(125511):
    if(df5["File_Name"][k] in matched_files):
      dr.append(doc_rep[k]) # storing relevant vectors 
    else:
      ndr.append(doc_rep[k]) # storing non relevant vectors
  
  dr_arr=np.array(dr) # conversion into array
  ndr_arr=np.array(ndr)
  dr_rev[i]=np.average(dr_arr,axis=0) #finding average of all relevant doc vectors
  ndr_rev[i]=np.average(ndr_arr,axis=0)

processing
processing
processing
processing


/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing


In [0]:
print(dr_rev[4])

[ 0.17489338  0.165007   -0.18037026 -0.06686243  0.40938255  0.24050386
  0.05484399 -0.01986844  0.60559773 -0.68563646 -0.13316394 -0.2776184
  0.05751168  0.29554704 -0.00387559  0.26320407  0.22360873 -0.17462482
 -0.1753766   0.11892575  0.13384472 -0.00600607  0.4074545  -0.02290263
  0.03623313 -0.23599319 -0.30831888 -0.12849821 -0.18349868  0.350342
 -0.21881266 -0.21352512  0.09438553  0.11115818  0.00525315  0.2168932
 -0.01641205  0.66164035  0.13384831 -0.09364513 -0.10627553 -0.32285216
 -0.27133974 -0.02120701  0.3265643  -0.37005582 -0.40882444  0.16829258
 -0.01503256 -0.2530879  -0.05947053 -0.14065918 -0.34453514 -0.36982927
  0.07381889 -0.193138   -0.12315749  0.03013054 -0.3467801   0.11666399
 -0.15175796  0.37923217  0.04777327 -0.10443452 -0.18058433  0.5571843
 -0.18302847 -0.16344418  0.17697395 -0.43335828  0.23316522 -0.18062298
 -0.20108889  0.19607848 -0.0639543   0.3073893  -0.08353105 -0.44624075
  0.14886652 -0.2171393   0.33865848 -0.3486105   0.0797

ROCCHIO Algorithm implementation

In [0]:
beta=0.75
gamma=0
mod_q=np.zeros((50,100),dtype='float32')
for i in range(50):
  if(len(list(set(rel_docs[i]) & set(actual_rel_docs[i])))>0): # checking no. of matches > 0 or not
    total_rel=len(list(set(rel_docs[i]) & set(actual_rel_docs[i])));
    total_non_rel=125511-total_rel
    term=beta/total_rel;
    mod_q[i]=np.add(query_rep[i],term*dr_rev[i])
  else:
    mod_q[i]=query_rep[i]


In [0]:
print(query_rep[1])

[-0.17854245  0.12555636 -0.35071468 -0.08304387  0.30471975  0.07430983
  0.24992079  0.19946437  0.82482666 -0.67209965 -0.12776779 -0.3130162
  0.49756286  0.17624891 -0.26101702  0.3335806   0.15055116 -0.03607667
 -0.38505483  0.21868862 -0.03247211  0.02342695  0.32564238  0.25632682
  0.27206585 -0.24816124 -0.19837065  0.03843203  0.09454639  0.27066457
 -0.39063284  0.02666064  0.07926484  0.01865079  0.22574836  0.12018254
  0.13322885  0.23490095  0.06448142 -0.4727353   0.33408478 -0.5770214
 -0.25758442  0.2848422   0.01275477 -0.61499196 -0.13797621  0.19366394
  0.08383601 -0.11163533  0.01836526 -0.34223303 -0.32481498 -0.25460246
  0.00906034 -0.49969435 -0.5433059   0.22994147 -0.0316832   0.17462033
 -0.16950904  0.5696799  -0.0682418  -0.36653912 -0.2636133   0.33851478
 -0.3309891  -0.16808383 -0.03129688  0.02096666  0.44607434 -0.8118524
 -0.12935837  0.34888396 -0.24145143  0.15798569  0.12613405 -0.4701724
  0.08468159  0.07197347  0.18541443 -0.21821357  0.192

In [0]:
print(mod_q[1])

[-1.88759029e-01  1.42829001e-01 -3.74821097e-01 -9.46791768e-02
  3.47334862e-01  1.03676453e-01  2.76318669e-01  2.14544401e-01
  9.06726718e-01 -7.64554501e-01 -1.41038179e-01 -3.62241864e-01
  5.14188468e-01  2.13741213e-01 -2.85478055e-01  3.71384650e-01
  1.82030216e-01 -5.17089963e-02 -4.17333215e-01  2.44845629e-01
 -1.96102783e-02  4.07879613e-02  3.73430848e-01  2.77852535e-01
  2.88715541e-01 -2.85537004e-01 -2.66004205e-01  4.10807692e-02
  7.98154399e-02  3.01225215e-01 -4.19256538e-01 -5.05834818e-04
  1.01103030e-01  6.90104160e-03  2.36267000e-01  1.46759182e-01
  1.41248778e-01  3.14407647e-01  8.03027377e-02 -4.95852381e-01
  3.48489970e-01 -6.30953729e-01 -2.83209294e-01  2.84536690e-01
  3.42602804e-02 -6.95031166e-01 -1.78478047e-01  2.17415974e-01
  9.51353386e-02 -1.45327672e-01  1.38374604e-02 -3.86085153e-01
 -3.72249842e-01 -2.93456793e-01  1.72169246e-02 -5.66506863e-01
 -5.83090782e-01  2.54438490e-01 -3.95362154e-02  2.04888865e-01
 -1.82662800e-01  6.25907

Cosine Similarity using Modified Query

In [0]:
#finding relevant with the modified query

from sklearn.metrics.pairwise import cosine_similarity
cos_lib2 = cosine_similarity(mod_q,doc_rep)

In [0]:
print(cos_lib2.shape)

(50, 125511)


In [0]:
dist2=[]
for i in range(len(cos_lib2[0])):
  dist2.append((i,cos_lib2[0][i]))

dist2=sorted(dist,key=lambda x:x[1],reverse=True)

Calculation of final relevant docs for each query

In [0]:
rel_docs2=[] # to store relev
p=0;
for k in range(0,50):
  dist2=[]
  a2=[] # to store relev
  b2=[] # actual rel
  for i in range(len(cos_lib2[k])):
    dist2.append((i,cos_lib2[k][i]))

  dist2=sorted(dist2,key=lambda x:x[1],reverse=True)
  print("Processing...");
  i=0;
  while(dist2[i][1]>=0.83 and i<125511):
   # print(dist[i]," ",df3["File_Name"][dist[i][0]].split('\\')[-1])
    a2.append(df3["File_Name"][dist2[i][0]].split('\\')[-1])
    i=i+1;
  
  rel_docs2.append(a2)

  print("------------------------------------")

Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
-----------------

In [0]:
matches=0
for i in range(50):
  matches=matches+len(set(rel_docs2[i]) & set(actual_rel_docs[i]))

print("Total Matches: ",matches)

Total Matches:  320


As we can see, no. of matches increased from 191 to 320 after implementing Rocchio Algorithm

In [0]:
print("Average Precison: ", matches/125511)
print("Average Recall: ", matches/653)

Average Precison:  0.0025495773278836117
Average Recall:  0.4900459418070444
